# # 🏔️ Stable Cascade Quickstart
# 
# **學習目標**: 掌握 Stable Cascade 雙階段架構推論，理解與 SD 的差異，實作低 VRAM 優化策略
# 
# **Cascade 特色**:
# - 💎 雙階段設計：Stage B (壓縮) + Stage C (生成)
# - 🖼️ 原生高解析度支援 (1024x1024+)  
# - 🎯 更好的文本-圖像對齊
# - ⚡ 相對高效的推論速度


In [ ]:
# %% [1] Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# %%
# 📦 Package imports & environment validation
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

import torch
import torch.nn as nn
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline
from diffusers.utils import load_image
import numpy as np
from PIL import Image
import time
import gc
from typing import Optional, List, Tuple
import json

# Environment flags
SMOKE_MODE = os.getenv("SMOKE_MODE", "false").lower() == "true"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"🚀 Environment: {DEVICE} | Precision: {DTYPE} | Smoke Mode: {SMOKE_MODE}")

In [ ]:
# %%
# 🏗️ Cascade Architecture Overview & Model Loading
print("📚 Stable Cascade Architecture:")
print("Stage C (Prior): Text → Latent Space (24x24)")
print("Stage B (Decoder): Latent (24x24) → High-res Image (1024x1024)")
print("\n🔽 Loading Cascade pipelines...")

# Prior pipeline (Stage C) - text to latent
prior_pipeline = StableCascadePriorPipeline.from_pretrained(
    "stabilityai/stable-cascade-prior",
    torch_dtype=DTYPE,
    variant="bf16" if DTYPE == torch.bfloat16 else "fp16",
)

# Decoder pipeline (Stage B) - latent to image
decoder_pipeline = StableCascadeDecoderPipeline.from_pretrained(
    "stabilityai/stable-cascade",
    torch_dtype=DTYPE,
    variant="bf16" if DTYPE == torch.bfloat16 else "fp16",
)

print("✅ Cascade pipelines loaded successfully")

In [ ]:
# %%
# ⚙️ Low-VRAM optimization strategies
def setup_low_vram_cascade(prior_pipe, decoder_pipe, enable_optimizations=True):
    """Apply memory optimizations for 8-12GB VRAM"""
    if not enable_optimizations:
        return prior_pipe, decoder_pipe

    print("🔧 Applying low-VRAM optimizations...")

    # Enable attention slicing (reduces VRAM usage)
    prior_pipe.enable_attention_slicing()
    decoder_pipe.enable_attention_slicing()

    # Enable memory efficient attention (xFormers)
    try:
        prior_pipe.enable_xformers_memory_efficient_attention()
        decoder_pipe.enable_xformers_memory_efficient_attention()
        print("✅ xFormers memory efficient attention enabled")
    except Exception as e:
        print(f"⚠️ xFormers not available: {e}")

    # Sequential CPU offload for very low VRAM (< 8GB)
    vram_gb = torch.cuda.get_device_properties(0).total_memory // (1024**3)
    if vram_gb < 10:
        print(f"🔄 VRAM ({vram_gb}GB < 10GB) - enabling sequential CPU offload")
        prior_pipe.enable_sequential_cpu_offload()
        decoder_pipe.enable_sequential_cpu_offload()
    else:
        # Model CPU offload for moderate VRAM
        prior_pipe.enable_model_cpu_offload()
        decoder_pipe.enable_model_cpu_offload()

    return prior_pipe, decoder_pipe


# Apply optimizations
prior_pipeline, decoder_pipeline = setup_low_vram_cascade(
    prior_pipeline, decoder_pipeline, enable_optimizations=True
)

In [ ]:
# %%
# 🚀 Minimal Working Example (MVP)
def generate_cascade_image(
    prompt: str,
    negative_prompt: str = "",
    height: int = 1024,
    width: int = 1024,
    prior_guidance_scale: float = 4.0,
    decoder_guidance_scale: float = 0.0,
    num_inference_steps_prior: int = 20,
    num_inference_steps_decoder: int = 10,
    seed: Optional[int] = None,
) -> Tuple[Image.Image, dict]:
    """
    Generate image using Stable Cascade two-stage process

    Returns:
        tuple: (generated_image, metadata_dict)
    """
    # Set seed for reproducibility
    generator = torch.Generator(device=DEVICE)
    if seed is not None:
        generator.manual_seed(seed)

    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    start_time = time.time()

    # Stage C: Text to latent (Prior)
    print(f"🎯 Stage C (Prior): '{prompt[:50]}...'")
    prior_output = prior_pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=height,
        width=width,
        guidance_scale=prior_guidance_scale,
        num_inference_steps=num_inference_steps_prior,
        generator=generator,
    )

    # Stage B: Latent to image (Decoder)
    print("🖼️ Stage B (Decoder): Latent → High-res image")
    decoder_output = decoder_pipeline(
        image_embeddings=prior_output.image_embeddings,
        prompt=prompt,
        negative_prompt=negative_prompt,
        guidance_scale=decoder_guidance_scale,
        num_inference_steps=num_inference_steps_decoder,
        generator=generator,
    )

    elapsed_time = time.time() - start_time

    # Prepare metadata
    metadata = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "dimensions": f"{width}x{height}",
        "prior_steps": num_inference_steps_prior,
        "decoder_steps": num_inference_steps_decoder,
        "prior_cfg": prior_guidance_scale,
        "decoder_cfg": decoder_guidance_scale,
        "seed": seed,
        "generation_time": round(elapsed_time, 2),
        "model": "stable-cascade",
    }

    print(f"✅ Generated in {elapsed_time:.2f}s")
    return decoder_output.images[0], metadata


# MVP Example
prompt = "a majestic mountain landscape with aurora borealis, oil painting style, highly detailed"
image, meta = generate_cascade_image(
    prompt=prompt,
    height=1024 if not SMOKE_MODE else 512,
    width=1024 if not SMOKE_MODE else 512,
    num_inference_steps_prior=20 if not SMOKE_MODE else 5,
    num_inference_steps_decoder=10 if not SMOKE_MODE else 3,
    seed=42,
)

print(f"📊 Generation metadata: {json.dumps(meta, indent=2)}")
image.save("cascade_mountain_aurora.png")
print("💾 Saved: cascade_mountain_aurora.png")

In [ ]:
# %%
# 🎯 Parameter exploration & comparison
def cascade_parameter_study():
    """Compare different parameter settings for Cascade"""

    base_prompt = "a serene japanese garden with cherry blossoms"
    configs = (
        [
            {"name": "Fast", "prior_steps": 10, "decoder_steps": 5, "prior_cfg": 2.0},
            {
                "name": "Balanced",
                "prior_steps": 20,
                "decoder_steps": 10,
                "prior_cfg": 4.0,
            },
            {
                "name": "Quality",
                "prior_steps": 30,
                "decoder_steps": 15,
                "prior_cfg": 6.0,
            },
        ]
        if not SMOKE_MODE
        else [{"name": "Smoke", "prior_steps": 3, "decoder_steps": 2, "prior_cfg": 2.0}]
    )

    results = []

    for config in configs:
        print(f"\n🧪 Testing {config['name']} preset...")

        image, meta = generate_cascade_image(
            prompt=base_prompt,
            height=768 if not SMOKE_MODE else 512,
            width=768 if not SMOKE_MODE else 512,
            num_inference_steps_prior=config["prior_steps"],
            num_inference_steps_decoder=config["decoder_steps"],
            prior_guidance_scale=config["prior_cfg"],
            seed=123,
        )

        # Save comparison image
        filename = f"cascade_garden_{config['name'].lower()}.png"
        image.save(filename)

        results.append(
            {
                "preset": config["name"],
                "time": meta["generation_time"],
                "prior_steps": config["prior_steps"],
                "decoder_steps": config["decoder_steps"],
                "filename": filename,
            }
        )

    # Print comparison table
    print("\n📈 Parameter Study Results:")
    print("Preset    | Time(s) | Prior Steps | Decoder Steps | Quality Notes")
    print("----------|---------|-------------|---------------|---------------")
    for r in results:
        print(
            f"{r['preset']:<9} | {r['time']:<7} | {r['prior_steps']:<11} | {r['decoder_steps']:<13} | See {r['filename']}"
        )

    return results


study_results = cascade_parameter_study()

In [ ]:
# %%
# 📊 Cascade vs SD benchmark comparison
def cascade_vs_sd_benchmark():
    """Quick benchmark comparison between Cascade and SD"""

    test_prompt = "a cyberpunk cityscape at night, neon lights, detailed"

    print("🏁 Cascade vs SD Benchmark")
    print("=" * 50)

    # Cascade timing (already loaded)
    cascade_start = time.time()
    cascade_img, cascade_meta = generate_cascade_image(
        prompt=test_prompt,
        height=768 if not SMOKE_MODE else 512,
        width=768 if not SMOKE_MODE else 512,
        num_inference_steps_prior=15 if not SMOKE_MODE else 3,
        num_inference_steps_decoder=8 if not SMOKE_MODE else 2,
        seed=999,
    )
    cascade_time = time.time() - cascade_start

    # For fair comparison, we'd load SD here (commented out for this demo)
    # sd_pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
    sd_time_estimate = "~8-12s"  # Typical SD1.5 timing for reference

    print(f"\n⚡ Performance Comparison:")
    print(f"Cascade: {cascade_time:.2f}s @ {cascade_meta['dimensions']}")
    print(f"SD 1.5:  {sd_time_estimate} @ 512x512 (estimated)")
    print(f"\n🎨 Quality Notes:")
    print("- Cascade: Better text alignment, native high-res")
    print("- SD 1.5: Faster inference, wider ecosystem support")

    cascade_img.save("cascade_cyberpunk_benchmark.png")

    return {
        "cascade_time": cascade_time,
        "cascade_resolution": cascade_meta["dimensions"],
        "sd_estimate": sd_time_estimate,
    }


benchmark_results = cascade_vs_sd_benchmark()

In [ ]:
# %%
# 🧪 Smoke test (CI-compatible)
def smoke_test_cascade():
    """Minimal test for CI/automated testing"""
    print("🧪 Running Cascade smoke test...")

    # Ultra-minimal generation
    test_img, test_meta = generate_cascade_image(
        prompt="a red apple",
        height=512,
        width=512,
        num_inference_steps_prior=3,
        num_inference_steps_decoder=2,
        seed=1337,
    )

    # Validate output
    assert test_img.size == (512, 512), f"Expected 512x512, got {test_img.size}"
    assert (
        test_meta["generation_time"] < 30
    ), f"Too slow: {test_meta['generation_time']}s"

    test_img.save("cascade_smoke_test.png")
    print("✅ Smoke test passed!")

    return test_img, test_meta


if SMOKE_MODE:
    smoke_img, smoke_meta = smoke_test_cascade()

In [ ]:
# %%
# 💾 Save configuration and cleanup
cascade_config = {
    "model_name": "stable-cascade",
    "prior_model": "stabilityai/stable-cascade-prior",
    "decoder_model": "stabilityai/stable-cascade",
    "dtype": str(DTYPE),
    "device": DEVICE,
    "optimizations": {
        "attention_slicing": True,
        "xformers": True,
        "cpu_offload": (
            "sequential"
            if torch.cuda.get_device_properties(0).total_memory // (1024**3) < 10
            else "model"
        ),
    },
    "recommended_params": {
        "prior_steps": 20,
        "decoder_steps": 10,
        "prior_guidance_scale": 4.0,
        "decoder_guidance_scale": 0.0,
        "resolution": "1024x1024",
    },
    "benchmark_results": benchmark_results,
}

# Save config for future reference
with open("cascade_config.json", "w") as f:
    json.dump(cascade_config, f, indent=2)

print("💾 Configuration saved to cascade_config.json")

# Cleanup memory
del prior_pipeline, decoder_pipeline
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("🧹 Memory cleanup completed")

# ## 📋 Stage Summary
# 
# ### ✅ Completed
# - 🏗️ Stable Cascade 雙階段架構理解與實作
# - ⚙️ 8-12GB VRAM 低記憶體優化策略
# - 🎯 參數調優與品質/速度權衡分析
# - 📊 Cascade vs SD 基準比較
# - 🧪 CI-compatible 煙霧測試
# 
# ### 🧠 Key Concepts
# - **雙階段設計**: Stage C (text→latent) + Stage B (latent→image)
# - **原生高解析度**: 直接支援 1024x1024+ 無需額外超分
# - **記憶體管理**: attention slicing + CPU offload 策略
# - **參數權衡**: prior_steps vs decoder_steps 的速度/品質平衡
# 
# ### ⚠️ Common Pitfalls  
# - decoder_guidance_scale 通常設為 0.0（與 SD 不同）
# - 雙 pipeline 載入需要更多 VRAM（需要 CPU offload）
# - Stage C 的 guidance_scale 比 SD 建議值更低 (2.0-6.0)
# 
# ### 🎯 Next Steps
# - **ControlNet**: Cascade 條件控制實作（若支援）
# - **Fine-tuning**: Stage B/C 的 LoRA 微調探索
# - **Img2Img**: Cascade 圖生圖流程實作
# - **Pipeline Integration**: 與 SD 混合使用策略